# Proyecto 2 – Modelos de clasificación supervisada para evaluar la calidad de un automovil

### Integrantes
- Camilo Andres Galeano Trujillo

### Problema
Por medio de una base de datos con información referente a automoviles, se quiere clasificar la calidad de cada uno de los modelos, teniendo en cuenta las siguientes variables categóricas ordinales:

- **buying**: Precio de compra. Posibles valores: **(vhigh, high, med, low)**
- **maint**: Costo de mantenimiento. Posibles valores: **(vhigh, high, med, low)**
- **doors**: Número de puertas. Posibles valores: **(2, 3, 4, 5more)**
- **persons**: Cantidad de personas que puede llevar. Posibles valores: **(2, 4, more)**
- **lug_boot**: Capacidad para equipaje. Posibles valores: **(small, med, big)**
- **safety**: Seguridad estimada. Posibles valores: **(low, med, high)**
- **class**: Clasificación de evaluación. Posibles valores: **(unacc, acc, good, vgood)**

### Enlace GitHub

<https://github.com/cgatrujillo/ml-ean>

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

In [7]:
df = pd.read_csv("car.data", sep=",", names=["buying", "maint", "doors", "persons", "lug_boot", "safety", "class"])
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1728 entries, 0 to 1727
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   buying    1728 non-null   object
 1   maint     1728 non-null   object
 2   doors     1728 non-null   object
 3   persons   1728 non-null   object
 4   lug_boot  1728 non-null   object
 5   safety    1728 non-null   object
 6   class     1728 non-null   object
dtypes: object(7)
memory usage: 94.6+ KB


In [29]:
df["class"].value_counts()

class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64

In [9]:
df.describe()

,buying,maint,doors,persons,lug_boot,safety,class
count,1728,1728,1728,1728,1728,1728,1728
unique,4,4,4,3,3,3,4
top,vhigh,vhigh,2,2,small,low,unacc
freq,432,432,432,576,576,576,1210


In [11]:
df_ordinal = df.copy()

In [12]:
df_ordinal['buying'] = df_ordinal['buying'].map({'vhigh': 1, 'high': 2, 'med': 3, 'low':4})
df_ordinal['maint'] = df_ordinal['maint'].map({'vhigh': 1, 'high': 2, 'med': 3, 'low':4})
df_ordinal['doors'] = df_ordinal['doors'].map({'2': 1, '3': 2, '4': 3, '5more':4})
df_ordinal['class'] = df_ordinal['class'].map({'unacc': 1, 'acc': 2, 'good': 3, 'vgood':4})
df_ordinal['persons'] = df_ordinal['persons'].map({'2': 1, '4': 2, 'more': 3})
df_ordinal['lug_boot'] = df_ordinal['lug_boot'].map({'small': 1, 'med': 2, 'big': 3})
df_ordinal['safety'] = df_ordinal['safety'].map({'low': 1, 'med': 2, 'high': 3})

In [15]:
X = df_ordinal.drop(columns="class")
y = df_ordinal['class']

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 5)

In [16]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', None)
])

In [30]:
param_grid = [
    {
        'clf': [SVC()],
        'clf__C': [0.1, 1, 10],
        'clf__gamma': [0.01, 0.1, 1]
    },
    {
        'clf': [RandomForestClassifier()],
        'clf__n_estimators': [50, 100, 200],
        'clf__max_depth': [None, 5, 10]
    },
    {
        'clf': [LogisticRegression()],
        'clf__C': [0.1, 1, 10],
        'clf__penalty': ['l2', None]
    }
]

In [ ]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring = 'f1_weighted')
grid_search.fit(Xtrain, ytrain)

In [32]:
print("Mejores parámetros encontrados:", grid_search.best_params_)

Mejores parámetros encontrados: {'clf': SVC(), 'clf__C': 10, 'clf__gamma': 0.1}


In [24]:
grid_search.best_score_

0.9774483330164117

In [25]:
best_clf = grid_search.best_estimator_
best_clf.fit(Xtrain, ytrain)
y_pred = best_clf.predict(Xtest)

In [26]:
accuracy_score(ytest, y_pred)

0.9884393063583815

In [28]:
print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       242
           2       0.99      0.99      0.99        77
           3       0.92      0.86      0.89        14
           4       0.93      1.00      0.96        13

    accuracy                           0.99       346
   macro avg       0.96      0.96      0.96       346
weighted avg       0.99      0.99      0.99       346



En nuestro caso el mejor modelo fue un Support Vector Machine, consiguiendo una respuesta con la validación cruzada de 0.98 de f1-score promedio ponderado, a parte de eso se vió la respuesta en cada una de las clasificaciones, teniendo un poco de dificultades al clasificar los modelos con clasificación buena, de todas formas un posible problema que puede tener el modelo a la larga es la poca información que se tiene sobre los modelos buenos y muy buenos. El desbalance de clases es algo que se puede tener en cuenta a la larga.

Por otra parte se recuerda que se realizaron pruebas con los modelos Support Vector Machine, Random Forest y Logistic Regression.